## 앰비언트 에이전트 구축 소개

이 과정의 주요 목표는 앰비언트 에이전트의 개념과 기존 채팅 에이전트와의 차이점을 탐구하는 것입니다.

## LangGraph의 핵심 기본 사항

이 과정은 노드, 엣지, 상태 그래프와 같은 LangGraph의 핵심 기본 사항에 익숙해지도록 설계되었습니다. 이러한 요소들은 우리가 만들 이메일 에이전트의 핵심 구성 요소가 될 것입니다.

다양한 도구 및 플랫폼과의 통합 방법을 시연할 예정입니다. 여기에는 다양한 모델 제공업체와의 통합을 위한 LangChain, 관찰 가능성 및 평가를 위한 LangSmith, 배포를 위한 LangGraph 플랫폼, 시각화 및 디버깅을 위한 LangGraph Studio 사용법이 포함됩니다.

![](./assets/36051.png)

## Resources

- Notebook Reference: langgraph_101.ipynb
- Download Notebook on [GitHub](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/langgraph_101.ipynb)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - LangGraph 101.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/b9a/35b/890/Building_Ambient_Agents_with_LangGraph_-_LangGraph_101.pdf)

## 환경 설정

In [ ]:
import os

from dotenv import load_dotenv


load_dotenv("../../../.env", override=True)

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

False

## Chat models